# $$\color{red}{\text{Artificial Intelligence - Camputer Assignment 2}}$$
                                                            Part 2: Game
$$\color{lime}{\text{Alireza Javid - 810099011}}$$

## $\color{deepskyblue}{\text{Introduction}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این تمرین با استفاده از روش های adversial search سعی در بیشتر کردن احتمال برد مان در یک بازی می کنیم.</font>
</p>

## $\color{deepskyblue}{\text{Import Libraries}}$

In [4]:
from select import select
from symbol import dotted_as_name
import turtle
import math
import random
from time import sleep
from sys import argv
import copy
from time import time
from collections import Counter

C:\Users\hooshmand\AppData\Local\Temp\ipykernel_30224\1573988184.py:2: DeprecationWarning: The symbol module is deprecated and will be removed in future versions of Python
  from symbol import dotted_as_name


## $\color{deepskyblue}{\text{Assign Parameters}}$

In [5]:
DUP_SCORE = -100
WIN_SCORE = 1000

## $\color{deepskyblue}{\text{Modeling}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تابع ارزیابی را به گونه ای می نویسیم که در صورتی که شکست تشخیص داد امتیاز منفی بالایی دهد و اگر 2 ضلع در یک نود مشترک بودند به دلیل آنکه احتمال تشکیل مثلث بالا می رود امتیاز منفی مناسبی دهد.
</font>
</p>

In [19]:
class Sim:
    # Set true for graphical interface
    GUI = False
    screen = None
    selection = []
    turn = ''
    dots = []
    red = []
    blue = []
    available_moves = []
    minimax_depth = 0
    prune = False

    def __init__(self, minimax_depth, prune, gui):
        self.GUI = gui
        self.prune = prune
        self.minimax_depth = minimax_depth
        if self.GUI:
            self.setup_screen()

    def setup_screen(self):
        self.screen = turtle.Screen()
        self.screen.setup(800, 800)
        self.screen.title("Game of SIM")
        self.screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self.screen.tracer(0, 0)
        turtle.hideturtle()

    def draw_dot(self, x, y, color):
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def gen_dots(self):
        r = []
        for angle in range(0, 360, 60):
            r.append((math.cos(math.radians(angle)), math.sin(math.radians(angle))))
        return r

    def initialize(self):
        self.selection = []
        self.available_moves = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    self.available_moves.append((i, j))
        if random.randint(0, 2) == 1:
            self.turn = 'red'
        else:
            self.turn = 'blue'
        self.dots = self.gen_dots()
        self.red = []
        self.blue = []
        if self.GUI: turtle.clear()
        self.draw()

    def draw_line(self, p1, p2, color):
        turtle.up()
        turtle.pensize(3)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def draw_board(self):
        for i in range(len(self.dots)):
            if i in self.selection:
                self.draw_dot(self.dots[i][0], self.dots[i][1], self.turn)
            else:
                self.draw_dot(self.dots[i][0], self.dots[i][1], 'dark gray')

    def draw(self):
        if not self.GUI: return 0
        self.draw_board()
        for i in range(len(self.red)):
            self.draw_line((math.cos(math.radians(self.red[i][0] * 60)), math.sin(math.radians(self.red[i][0] * 60))),
                           (math.cos(math.radians(self.red[i][1] * 60)), math.sin(math.radians(self.red[i][1] * 60))),
                           'red')
        for i in range(len(self.blue)):
            self.draw_line((math.cos(math.radians(self.blue[i][0] * 60)), math.sin(math.radians(self.blue[i][0] * 60))),
                           (math.cos(math.radians(self.blue[i][1] * 60)), math.sin(math.radians(self.blue[i][1] * 60))),
                           'blue')
        self.screen.update()
        sleep(1)

    def _evaluate(self):
        score = 0
        if self.winner == "red":
            score += WIN_SCORE
            return score
        elif self.winner == "blue":
            score += -WIN_SCORE
            return score
        dup_red = [k for k, v in Counter([item for t in self.red for item in t]).items() if v > 1]
        if dup_red:
            score += DUP_SCORE * len(dup_red)
        dup_blue = [k for k, v in Counter([item for t in self.blue for item in t]).items() if v > 1]
        if dup_blue:
            score += -DUP_SCORE * len(dup_blue)
        return score

    # TODO

    def minimax(self, depth, player_turn):
        # TODO
        self.winner = self.gameover(self.red, self.blue)
        if depth == 0 or (not self.available_moves) or self.winner:
            return None, self._evaluate()
        if player_turn == 'red':
            score = -math.inf
            for move in self.available_moves:
                self.red.append(move)
                self.available_moves.remove(move)
                new_score = self.minimax(depth - 1, 'blue')[1]
                self.red.remove(move)
                self.available_moves.append(move)
                if new_score > score:
                    score = new_score
                    selected_move = move
        else:
            score = math.inf
            for move in self.available_moves:
                self.blue.append(move)
                self.available_moves.remove(move)
                new_score = self.minimax(depth - 1, 'red')[1]
                self.blue.remove(move)
                self.available_moves.append(move)
                if new_score < score:
                    score = new_score
                    selected_move = move
        return selected_move, score

    def pruning_maximin(self, depth, player_turn, beta, alpha):
        self.winner = self.gameover(self.red, self.blue)
        if depth == 0 or (not self.available_moves) or self.winner:
            return None, self._evaluate()
        random.shuffle(self.available_moves)
        if player_turn == 'red':
            best_score = -math.inf
            for move in self.available_moves:
                self.red.append(move)
                self.available_moves.remove(move)
                new_score = self.pruning_maximin(depth - 1, 'blue', beta, alpha)[1]
                self.red.remove(move)
                self.available_moves.append(move)
                if new_score > best_score:
                    best_score = new_score
                    selected_move = move
                best_score = max(new_score, best_score)
                alpha = max(best_score, alpha)
                if alpha >= beta:
                    break
        else:
            best_score = math.inf
            for move in self.available_moves:
                self.blue.append(move)
                self.available_moves.remove(move)
                new_score = self.pruning_maximin(depth - 1, 'red', beta, alpha)[1]
                self.blue.remove(move)
                self.available_moves.append(move)
                if new_score < best_score:
                    best_score = new_score
                    selected_move = move
                best_score = min(new_score, best_score)
                beta = min(best_score, beta)
                if alpha >= beta:
                    break
        return selected_move, best_score





    def enemy(self):
        return random.choice(self.available_moves)

    def play(self, pruning=None):
        self.initialize()
        while True:
            if self.turn == 'red':
                if pruning:
                    selection = self.pruning_maximin(depth=self.minimax_depth, player_turn=self.turn,
                                                beta=math.inf, alpha=-math.inf)[0]
                else:
                    selection = self.minimax(depth=self.minimax_depth, player_turn=self.turn)[0]
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            else:
                selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            if selection in self.red or selection in self.blue:
                raise Exception("Duplicate Move!!!")
            if self.turn == 'red':
                self.red.append(selection)
            else:
                self.blue.append(selection)

            self.available_moves.remove(selection)
            self.turn = self._swap_turn(self.turn)
            selection = []
            self.draw()
            r = self.gameover(self.red, self.blue)
            if r != 0:
                return r

    def _swap_turn(self, turn):
        if turn == 'red':
            return 'blue'
        else:
            return 'red'

    def gameover(self, r, b):
        if len(r) < 3:
            return 0
        r.sort()
        for i in range(len(r) - 2):
            for j in range(i + 1, len(r) - 1):
                for k in range(j + 1, len(r)):
                    if r[i][0] == r[j][0] and r[i][1] == r[k][0] and r[j][1] == r[k][1]:
                        return 'blue'
        if len(b) < 3: return 0
        b.sort()
        for i in range(len(b) - 2):
            for j in range(i + 1, len(b) - 1):
                for k in range(j + 1, len(b)):
                    if b[i][0] == b[j][0] and b[i][1] == b[k][0] and b[j][1] == b[k][1]:
                        return 'red'
        return 0

## $\color{deepskyblue}{\text{Running Without Puring}}$

In [20]:
depths = [1, 3, 5]
for depth in depths:
    game = Sim(minimax_depth=depth, prune=True, gui=0)
    results = {'red': 0, 'blue': 0}
    start_time = time()
    for i in range(100):
    #print(i)
        results[game.play()] += 1
    finish_time = time()
    print('Results for depth = '+ str(depth))
    print(results)
    print("win probability = " + str(results['red'] / 100))
    print("Execution Time for depth = "+ str(depth) + " without puring : " + str(finish_time-start_time) + " seconds\n")

Results for depth = 1
{'red': 80, 'blue': 20}
win probability = 0.8
Execution Time for depth = 1 without puring : 0.042882442474365234 seconds

Results for depth = 3
{'red': 84, 'blue': 16}
win probability = 0.84
Execution Time for depth = 3 without puring : 3.026935577392578 seconds

Results for depth = 5
{'red': 89, 'blue': 11}
win probability = 0.89
Execution Time for depth = 5 without puring : 268.09166717529297 seconds



## $\color{deepskyblue}{\text{Running With Puring}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

</font>
</p>

In [24]:
depths = [1, 3, 5, 7]
for depth in depths:
    game = Sim(minimax_depth=depth, prune=True, gui=0)
    results = {'red': 0, 'blue': 0}
    start_time = time()
    for i in range(100):
    #print(i)
        results[game.play(pruning=True)] += 1
    finish_time = time()
    print('Results for depth = '+ str(depth))
    print(results)
    print("win probability = " + str(results['red'] / 100))
    print("Execution Time for depth = "+ str(depth) + " with puring : " + str(finish_time-start_time) + " seconds\n")

Results for depth = 1
{'red': 78, 'blue': 22}
win probability = 0.78
Execution Time for depth = 1 with puring : 0.04587841033935547 seconds

Results for depth = 3
{'red': 80, 'blue': 20}
win probability = 0.8
Execution Time for depth = 3 with puring : 0.6472680568695068 seconds

Results for depth = 5
{'red': 82, 'blue': 18}
win probability = 0.82
Execution Time for depth = 5 with puring : 7.778202295303345 seconds

Results for depth = 7
{'red': 85, 'blue': 15}
win probability = 0.85
Execution Time for depth = 7 with puring : 75.18795776367188 seconds




## $\color{deepskyblue}{\text{Answers}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<ul dir =rtl>
<ol>
  <li>
یک تابع heuristic خوب باید به استیت هایی که شانس برد ما در آن ها بیشتر از امتیاز بیشتری اختصاص دهد. علت انتخاب heuristic ذکر شده در بالا این است که این تابع جامعیت خوبی دارد و به حالات مختلف امتیازات مختلف می‌دهد. مثلا اگر شکست را تشخیص دهد امتیاز منفی بالایی می دهد و اگر 2 راس از یک نود عبور کند به دلیل افزایش احتمال تشکیل مثلث امتیاز منفی متناسبی می دهد.</li>
  <li>
بله همانطور که در درس دیدیم رابطه وجود دارد: با افزایش عمق جست و جو شانس برد بیشتر می‌شود ولی زمان اجرا و تعداد راس های دیده شده نیز به صورت نمایی افزایش می‌یابد.  </li>
  <li>
  بله  این ترتیب اهمیت دارد چون می‌تواند از دیدن راس های زیادی جلوگیری کند، در این سوال از تابع random.shuffle استفاده شده است, این شیوه کمک می کند تا حرکات جمع نشود و به نود های کمتری نیاز به بازدید باشد.
  </li>
</ol> 
</font>
</p>



## $\color{deepskyblue}{\text{Conclusion}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
برای افزایش شانس برد در یک بازی نیازی نیست تمامی حالات را در نظر بگیریم و با یک استراتژی درست و تعریف heuristic مناسب و استفاده از تکنیک هایی مانند هرس آلفا و بتا می‌توان بدون چک کردن تمامی حالات ,در زمان مناسب یک راه حل با شانس برد بالا ارائه داد. 
</font>
</p>